<a href="https://colab.research.google.com/github/nrtk1986/my_first/blob/master/%E7%9B%B4%E8%BF%91%E3%82%82%E5%90%AB%E3%82%81%E3%81%9F%E6%A0%AA%E4%BE%A1%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np

#スクレイピング用
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
#取得したい株価コード
kabu_code = '9984'

##**Stooqから直近以外のデータを取得**

In [3]:
df_stooq_tmp = web.DataReader(kabu_code + ".JP", "stooq").dropna()
df_stooq = df_stooq_tmp.reset_index()

In [4]:
df_stooq.head(3)

,Date,Open,High,Low,Close,Volume
0,2020-11-02,6755.0,6848.0,6681.0,6696.0,14611900
1,2020-10-30,6890.0,6918.0,6793.0,6793.0,14443100
2,2020-10-29,6900.0,6965.0,6868.0,6907.0,11047900


## **株探から直近のデータ取得**

In [5]:
def get_latest(code):
  try:
      tgt = 'https://kabutan.jp/stock/kabuka?code=' + str(code)
      html = urlopen(tgt)
      bsObj = BeautifulSoup(html, 'html.parser')
      table = bsObj.findAll('table', {'class':'stock_kabuka0'})[0]
      rows = table.findAll('tr')
      for row in rows:
          rec = []
          for cell in row.findAll(['td', 'th']):
              rec.append(cell.get_text())
          del rec[5:7]
          rec.insert(0, str(code) + '.JP')
      dish.append(rec)
      return 'Success'
  except Exception as e:
      return str(code) + ': ' + str(e)

dish = []
get_latest(kabu_code)

df_latest_tmp = pd.DataFrame(dish).rename(columns={0: 'code', 1: 'Date', 2:'Open', 3: 'High', 4: 'Low', 5:'Close', 6: 'Volume'})
df_latest = df_latest_tmp.drop('code', axis=1)

### **Dateを日付型に変更**

In [6]:
df_latest.applymap(type)

,Date,Open,High,Low,Close,Volume
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [7]:
df_latest['Date'] =  '20' + df_latest['Date']
df_latest['Date'] = pd.to_datetime(df_latest['Date'], format='%Y-%m-%d')

In [8]:
#df_latest.head()

In [9]:
#df_latest.dtypes

### **株価、取引量をint型に変換**

In [10]:
#カンマ区切りのカンマを除外
df_latest['Open']   = df_latest['Open'].str.replace(',','')
df_latest['High']   = df_latest['High'].str.replace(',','')
df_latest['Low']    = df_latest['Low'].str.replace(',','')
df_latest['Close']  = df_latest['Close'].str.replace(',','')
df_latest['Volume'] = df_latest['Volume'].str.replace(',','')
df_latest.head()

,Date,Open,High,Low,Close,Volume
0,2020-11-04,6600,6678,6467,6535,21179800


In [11]:
#int型に変換
df_latest['Open']   = df_latest['Open'].astype(int)
df_latest['High']   = df_latest['High'].astype(int)
df_latest['Low']    = df_latest['Low'].astype(int)
df_latest['Close']  = df_latest['Close'].astype(int)
df_latest['Volume'] = df_latest['Volume'].astype(int)
df_latest.dtypes

Date      datetime64[ns]
Open               int64
High               int64
Low                int64
Close              int64
Volume             int64
dtype: object

## **Stooqのデータと株探のデータを結合する**

In [12]:
#データを結合する
df = pd.concat([df_stooq, df_latest])

In [13]:
#Dateの古い順にソートする
df = df.sort_values('Date')

In [14]:
#indexを振りなおす
df.reset_index(inplace=True, drop=True)
df.tail()

,Date,Open,High,Low,Close,Volume
1212,2020-10-28,6906.0,7054.0,6872.0,7003.0,12797600
1213,2020-10-29,6900.0,6965.0,6868.0,6907.0,11047900
1214,2020-10-30,6890.0,6918.0,6793.0,6793.0,14443100
1215,2020-11-02,6755.0,6848.0,6681.0,6696.0,14611900
1216,2020-11-04,6600.0,6678.0,6467.0,6535.0,21179800


In [16]:
df.to_csv('SoftBank_20201104.csv')